In [1]:
import pandas as pd

In [10]:
samples = pd.read_csv('./bbn.csv', sep = ',', header = 0)
samples.head()

,Battery,Radio,Lights,Ignition,Gas,EngineStarts
0,1,1,1,1,1,1
1,1,1,1,0,1,0
2,1,0,1,1,1,1
3,1,0,1,1,1,0
4,0,1,0,0,1,0


In [19]:
samples.shape[0]

20

In [27]:
samples[samples.Battery == 0].shape[0]

3

In [26]:
samples[((samples.Lights == 1) & (samples.Battery == 0))].shape[0]

1

In [25]:
samples[((samples.Lights == 1) & (samples.Battery == 0))].shape[0]/samples[samples.Battery == 0].shape[0]

0.3333333333333333

In [35]:
samples[((samples.Ignition == 1) & (samples.Gas == 0))].shape[0]

4

In [36]:
samples[((samples.EngineStarts == 1) & (samples.Ignition == 1) & (samples.Gas == 0))].shape[0]

3

---
Tuning Hyperparameters

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Feb  3 18:17:41 2022

@author: milos
"""

# Problem 2. Part 1. Linear regression

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error

# load the boston housing dataset
housing = datasets.fetch_openml(name='boston',version=1,parser='auto')
housing.details 

X=housing.data
Y=housing.target

# train/test split
X_train=X[:-100]
X_test=X[-100:]
Y_train=Y[:-100]
Y_test=Y[-100:]

print(f"Train Input Dimensions: {X_train.shape}")
print(f"Test Input Dimensions: {X_test.shape}")

# scale X components
scaler = StandardScaler()
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

Train Input Dimensions: (406, 13)
Test Input Dimensions: (100, 13)


In [2]:
def stoch_grad_des(n_interations, learning_rate, X_train, Y_train):
    # number of iterations for fitting w/ stochastic gradient descent
    niter = n_interations
    tau = learning_rate # learning rate
    n_samples = X_train.shape[0] # number of samples in training set
    n_feats = X_train.shape[1] # number of features

    # +1 for intercept
    weights = np.ones(shape = n_feats + 1, dtype = np.float64)
    for t in range(1,niter+1):
        # annealed rate
        rate = tau/t
        train_index = (t-1) % n_samples
        b = weights[0] # intercept
        w = weights[1:] # slope coef

        # sample
        x = X_train[train_index,:]
        y = Y_train[train_index]

        # single sample, use dot product
        gradient = y - (w.dot(x) + b)
        weights[0] = b + rate*gradient
        weights[1:] = w + rate*gradient*x

    # Create linear regression object
    regr = linear_model.LinearRegression()
    regr.intercept_ = weights[0]
    regr.coef_ = weights[1:]

    return regr

In [7]:
best_params = {}
for params in np.linspace(0.05, 0.5, num = 100):
    regr = stoch_grad_des(10000, params, X_train, Y_train)
    Y_train_pred = regr.predict(X_train)
    error = mean_squared_error(Y_train, Y_train_pred)
    best_params[params] = error

In [8]:
best_params

{(1000, 0.1): 221.39790487606044,
 (2000, 0.1): 191.42617260587045,
 (5000, 0.1): 161.13420716876996,
 (10000, 0.1): 142.90367198098352,
 (1000, 0.2): 137.56527889564205,
 (2000, 0.2): 107.66550756485482,
 (5000, 0.2): 83.12317310037963,
 (10000, 0.2): 69.95432673865545,
 (1000, 0.3): 104.5343438552953,
 (2000, 0.3): 76.81907147413092,
 (5000, 0.3): 57.72110958256593,
 (10000, 0.3): 48.806154928746466,
 (1000, 0.4): 87.3838368725379,
 (2000, 0.4): 63.12952555813335,
 (5000, 0.4): 48.798914726435434,
 (10000, 0.4): 42.797458270157904,
 (1000, 0.5): 78.92902449663985,
 (2000, 0.5): 58.244537527998986,
 (5000, 0.5): 47.08226535270501,
 (10000, 0.5): 42.514132118930846}

In [ ]:
Y_train_pred = regr.predict(X_train)
print(f"MSE on Training Set: {round(mean_squared_error(Y_train, Y_train_pred),2)}")


# Make predictions on the testing set
Y_test_pred = regr.predict(X_test)
# The mean squared error on the test set
print('Mean squared testing error: %.2f'
      % mean_squared_error(Y_test, Y_test_pred))